# Dense subgraphs

| Student Name         | Student-ID |
|----------------------|------------|
| Marco Di Francesco   | 100632815  |
| Loreto García Tejada | 100643862  |
| György Bence Józsa   | 100633270  |
| József-Hunor Jánosi  | 100516724  |
| Sara-Jane Bittner    | 100498554  |

_Learning goal: To familiarize oneself with one definition of the density of subgraphs._

Consider a graph $G = (V, E)$. Given a subset of vertices $S \subseteq V$ , we propose the following
definition of density:

$$ D(S) = \frac{\sum_{i \in S} \sum_{j \in S} w(i,j)}{|S|}, $$

where $w(i, j)$ is the weight of the edge between vertices $i$ and $j$, if it exists, and $0$ otherwise,
and $|S|$ is the cardinality of the set $S$. Note that in unweighted graphs, $w(i, j) \in \{0, 1\}$.

In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def density(S: nx.Graph):
    """Calculates the density of (sub)graph S"""
    return S.size(weight='weight') / S.number_of_nodes()

a) Propose a greedy algorithm to find a subgraph with good density according to this definition. Explain the main properties of the algorithm, e.g., does the result depend on the order of execution etc.

Proposition:

We start with $S = \emptyset$.
In the first step, we choose the vertex $v_0$ such that the sum of the weights of the edges connecting to $v$ is maximal. We can call it the vertex with the maximal weight. Add that to $S$.
After that, in each step, we choose a vertex $v_i$ out of the neighbours of $S$, such that $D(S \cup v_i)$ is maximal, and add that to $S$. We repeat this step until $D(S) > D(S \cup v_i)$.
We return S.

In [37]:
def dense_subgraph(G: nx.Graph) -> nx.Graph:
    node_degrees = dict(G.degree)
    max_node = max(zip(node_degrees.values(), node_degrees.keys()))[1]
    S = [max_node]
    while True:
        neighbors = {x for n in S for x in G.neighbors(n)} # here be dragons
        max_value = 0
        candidate = -1
        for v in neighbors:
            S2 = S + [v]
            d1 = density(G.subgraph(S))
            d2 = density(G.subgraph(S2))
            if d2 > d1:
                if d2 > max_value:
                    max_value = d2
                    candidate = v
        if candidate == -1:
            return G.subgraph(S)
        S.append(candidate)


b) Load _dolphins.txt_ data set. In this file, the first line contains the number of vertices. Each subsequent line indicates the two endpoints of an **undirected** edge. Estimate some reasonable lower and upper bounds on $D$.

In [26]:
G = nx.Graph()
filename = 'dolphins.txt'
with open(filename) as f:
    n = int(next(f)) # first line is number of vertices
    for line in f:
        i, j = [int(x) for x in line.split()]
        G.add_edge(i, j)

num_edges = G.number_of_edges()

# minimum bound
num_nodes = num_edges * 2
min_bound = num_edges / num_nodes

# maximum bound
num_nodes = np.ceil((1 + np.sqrt(1 + 8 * num_edges)) / 2)
max_bound = num_edges / num_nodes

print(f'Minimum boundary of density D of {filename}: {min_bound}')
print(f'Maximum boundary of density D of {filename}: {max_bound}')

Minimum boundary of density D of dolphins.txt: 0.5
Maximum boundary of density D of dolphins.txt: 8.368421052631579


c) Apply your algorithm on the _dolphins.txt_ data. As a result, provide the maximum density you achieved and the vertices of the corresponding subgraph (in ascending order).

In [39]:
SG = dense_subgraph(G)

sorted_nodes = list(SG.nodes)
sorted_nodes.sort()
print(f'Density of subgraph: {density(SG)}')
print(f'Nodes in subgraph: {sorted_nodes}')

Density of subgraph: 3.0555555555555554
Nodes in subgraph: [1, 15, 16, 17, 19, 21, 22, 25, 30, 34, 38, 39, 41, 44, 46, 51, 52, 53]
[(1, 15), (1, 16), (1, 41), (34, 15), (34, 17), (34, 22), (34, 38), (34, 39), (34, 41), (34, 44), (34, 51), (38, 15), (38, 17), (38, 22), (38, 41), (38, 44), (38, 46), (39, 15), (39, 17), (39, 21), (39, 44), (39, 53), (41, 15), (41, 16), (41, 53), (44, 15), (44, 30), (46, 16), (46, 19), (46, 22), (46, 25), (46, 30), (46, 51), (46, 52), (15, 17), (15, 25), (15, 51), (15, 53), (16, 19), (16, 25), (17, 21), (17, 51), (19, 21), (19, 22), (19, 25), (19, 30), (19, 52), (52, 22), (52, 25), (52, 30), (52, 51), (51, 21), (22, 30), (53, 30), (25, 30)]


d) isualize the graph using Gephi ([https://gephi.org/](https://gephi.org/)). Use different coloring for vertices that are within the subgraph found in the previous step.

In [46]:
with open('dolphins2.csv', 'w') as f:
    f.write('Source,Target,Subgraph')
    for i, j in G.edges:
        if (i, j) in SG.edges:
            f.write(f'{i},{j},1\n')
        else:
            f.write(f'{i},{j},0\n')